# Simple recommender System

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
books = pd.read_csv('dataset/books.csv', low_memory=False)
books.head()

id  book_id  best_book_id  work_id  books_count       isbn        isbn13  \
0   1  2767052       2767052  2792775          272  439023483  9.780439e+12   
1   2        3             3  4640799          491  439554934  9.780440e+12   
2   3    41865         41865  3212258          226  316015849  9.780316e+12   
3   4     2657          2657  3275794          487   61120081  9.780061e+12   
4   5     4671          4671   245494         1356  743273567  9.780743e+12   

                       authors  original_publication_year  \
0              Suzanne Collins                     2008.0   
1  J.K. Rowling, Mary GrandPré                     1997.0   
2              Stephenie Meyer                     2005.0   
3                   Harper Lee                     1960.0   
4          F. Scott Fitzgerald                     1925.0   

                             original_title  ... ratings_count  \
0                          The Hunger Games  ...       4780653   
1  Harry Potter and the Philosopher's Stone  ...       4602479   
2                                  Twilight  ...       3866839   
3                     To Kill a Mockingbird  ...       3198671   
4                          The Great Gatsby  ...       2683664   

  work_ratings_count  work_text_reviews_count  ratings_1  ratings_2  \
0            4942365                   155254      66715     127936   
1            4800065                    75867      75504     101676   
2            3916824                    95009     456191     436802   
3            3340896                    72586      60427     117415   
4            2773745                    51992      86236     197621   

   ratings_3  ratings_4  ratings_5  \
0     560092    1481305    2706317   
1     455024    1156318    3011543   
2     793319     875073    1355439   
3     446835    1001952    1714267   
4     606158     936012     947718   

                                           image_url  \
0  https://images.gr-assets.com/books/1447303603m...   
1  https://images.gr-assets.com/books/1474154022m...   
2  https://images.gr-assets.com/books/1361039443m...   
3  https://images.gr-assets.com/books/1361975680m...   
4  https://images.gr-assets.com/books/1490528560m...   

                                     small_image_url  
0  https://images.gr-assets.com/books/1447303603s...  
1  https://images.gr-assets.com/books/1474154022s...  
2  https://images.gr-assets.com/books/1361039443s...  
3  https://images.gr-assets.com/books/1361975680s...  
4  https://images.gr-assets.com/books/1490528560s...  

[5 rows x 23 columns]

In [3]:
ratings = pd.read_csv('dataset/ratings.csv', low_memory=False)
ratings.head()

book_id  user_id  rating
0        1      314       5
1        1      439       3
2        1      588       5
3        1     1169       4
4        1     1185       4

In [4]:
tags = pd.read_csv('dataset/tags.csv', low_memory=False)
tags.head()

tag_id tag_name
0       0        -
1       1     --1-
2       2    --10-
3       3    --12-
4       4   --122-

In [5]:
book_tags = pd.read_csv('dataset/book_tags.csv', low_memory=False)
book_tags.head()

goodreads_book_id  tag_id   count
0                  1   30574  167697
1                  1   11305   37174
2                  1   11557   34173
3                  1    8717   12986
4                  1   33114   12716

In [6]:
to_read = pd.read_csv('dataset/to_read.csv', low_memory=False)
to_read.head()

user_id  book_id
0        1      112
1        1      235
2        1      533
3        1     1198
4        1     1874

In [7]:
books.columns

Index(['id', 'book_id', 'best_book_id', 'work_id', 'books_count', 'isbn',
       'isbn13', 'authors', 'original_publication_year', 'original_title',
       'title', 'language_code', 'average_rating', 'ratings_count',
       'work_ratings_count', 'work_text_reviews_count', 'ratings_1',
       'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5', 'image_url',
       'small_image_url'],
      dtype='object')

In [8]:
ratings.columns

Index(['book_id', 'user_id', 'rating'], dtype='object')

In [9]:
tags.columns

Index(['tag_id', 'tag_name'], dtype='object')

In [10]:
book_tags.columns

Index(['goodreads_book_id', 'tag_id', 'count'], dtype='object')

In [11]:
to_read.columns

Index(['user_id', 'book_id'], dtype='object')

# Clean data set

Remove Null value, drop the image_url,small_image_url columns

In [12]:
books = books.dropna()
books = books.drop(columns=['image_url', 'small_image_url'])

In [13]:
books.shape

(7860, 21)

In [14]:
#used to replace any missing or NaN values in the 'original_publication_year' column 
books['original_publication_year'] = books['original_publication_year'].fillna(-1).apply(lambda x: int(x) if x != -1 else -1)

In [15]:
#removes any duplicate rows from the ratings
ratings_rmv_duplicates = ratings.drop_duplicates()
#counts the number of rows in each group.
unwanted_users = ratings_rmv_duplicates.groupby('user_id')['user_id'].count()
#keeping only the users with a count of ratings less than 3.
unwanted_users = unwanted_users[unwanted_users < 3]
#keeping only the rows where the 'user_id' is an unwanted user
unwanted_ratings = ratings_rmv_duplicates[ratings_rmv_duplicates.user_id.isin(unwanted_users.index)]
#which has both duplicates and unwanted users removed, is assigned to a variable called new_ratings.
new_ratings = ratings_rmv_duplicates.drop(unwanted_ratings.index)

In [16]:
new_ratings.shape

(963473, 3)

In [17]:
new_ratings.head(10)

book_id  user_id  rating
0        1      314       5
1        1      439       3
2        1      588       5
3        1     1169       4
4        1     1185       4
5        1     2077       4
6        1     2487       4
7        1     2900       5
8        1     3662       4
9        1     3922       5

In [18]:
books.head(10)

id   book_id  best_book_id   work_id  books_count        isbn  \
0   1   2767052       2767052   2792775          272   439023483   
1   2         3             3   4640799          491   439554934   
2   3     41865         41865   3212258          226   316015849   
3   4      2657          2657   3275794          487    61120081   
4   5      4671          4671    245494         1356   743273567   
5   6  11870085      11870085  16827462          226   525478817   
6   7      5907          5907   1540236          969   618260307   
7   8      5107          5107   3036731          360   316769177   
8   9       960           960   3338963          311  1416524797   
9  10      1885          1885   3060926         3455   679783261   

         isbn13                      authors  original_publication_year  \
0  9.780439e+12              Suzanne Collins                       2008   
1  9.780440e+12  J.K. Rowling, Mary GrandPré                       1997   
2  9.780316e+12              Stephenie Meyer                       2005   
3  9.780061e+12                   Harper Lee                       1960   
4  9.780743e+12          F. Scott Fitzgerald                       1925   
5  9.780525e+12                   John Green                       2012   
6  9.780618e+12               J.R.R. Tolkien                       1937   
7  9.780317e+12                J.D. Salinger                       1951   
8  9.781417e+12                    Dan Brown                       2000   
9  9.780680e+12                  Jane Austen                       1813   

                             original_title  ... language_code average_rating  \
0                          The Hunger Games  ...           eng           4.34   
1  Harry Potter and the Philosopher's Stone  ...           eng           4.44   
2                                  Twilight  ...         en-US           3.57   
3                     To Kill a Mockingbird  ...           eng           4.25   
4                          The Great Gatsby  ...           eng           3.89   
5                    The Fault in Our Stars  ...           eng           4.26   
6        The Hobbit or There and Back Again  ...         en-US           4.25   
7                    The Catcher in the Rye  ...           eng           3.79   
8                          Angels & Demons   ...         en-CA           3.85   
9                       Pride and Prejudice  ...           eng           4.24   

   ratings_count  work_ratings_count  work_text_reviews_count  ratings_1  \
0        4780653             4942365                   155254      66715   
1        4602479             4800065                    75867      75504   
2        3866839             3916824                    95009     456191   
3        3198671             3340896                    72586      60427   
4        2683664             2773745                    51992      86236   
5        2346404             2478609                   140739      47994   
6        2071616             2196809                    37653      46023   
7        2044241             2120637                    44920     109383   
8        2001311             2078754                    25112      77841   
9        2035490             2191465                    49152      54700   

   ratings_2  ratings_3  ratings_4  ratings_5  
0     127936     560092    1481305    2706317  
1     101676     455024    1156318    3011543  
2     436802     793319     875073    1355439  
3     117415     446835    1001952    1714267  
4     197621     606158     936012     947718  
5      92723     327550     698471    1311871  
6      76784     288649     665635    1119718  
7     185520     455042     661516     709176  
8     145740     458429     716569     680175  
9      86485     284852     609755    1155673  

[10 rows x 21 columns]

In [19]:
bookMerge = books.merge(new_ratings,on='book_id')

In [20]:
bookMerge.head(150)

id  book_id  best_book_id  work_id  books_count       isbn        isbn13  \
0     2        3             3  4640799          491  439554934  9.780440e+12   
1     2        3             3  4640799          491  439554934  9.780440e+12   
2     2        3             3  4640799          491  439554934  9.780440e+12   
3     2        3             3  4640799          491  439554934  9.780440e+12   
4     2        3             3  4640799          491  439554934  9.780440e+12   
..   ..      ...           ...      ...          ...        ...           ...   
145   4     2657          2657  3275794          487   61120081  9.780061e+12   
146   4     2657          2657  3275794          487   61120081  9.780061e+12   
147   4     2657          2657  3275794          487   61120081  9.780061e+12   
148   4     2657          2657  3275794          487   61120081  9.780061e+12   
149   4     2657          2657  3275794          487   61120081  9.780061e+12   

                         authors  original_publication_year  \
0    J.K. Rowling, Mary GrandPré                       1997   
1    J.K. Rowling, Mary GrandPré                       1997   
2    J.K. Rowling, Mary GrandPré                       1997   
3    J.K. Rowling, Mary GrandPré                       1997   
4    J.K. Rowling, Mary GrandPré                       1997   
..                           ...                        ...   
145                   Harper Lee                       1960   
146                   Harper Lee                       1960   
147                   Harper Lee                       1960   
148                   Harper Lee                       1960   
149                   Harper Lee                       1960   

                               original_title  ... ratings_count  \
0    Harry Potter and the Philosopher's Stone  ...       4602479   
1    Harry Potter and the Philosopher's Stone  ...       4602479   
2    Harry Potter and the Philosopher's Stone  ...       4602479   
3    Harry Potter and the Philosopher's Stone  ...       4602479   
4    Harry Potter and the Philosopher's Stone  ...       4602479   
..                                        ...  ...           ...   
145                     To Kill a Mockingbird  ...       3198671   
146                     To Kill a Mockingbird  ...       3198671   
147                     To Kill a Mockingbird  ...       3198671   
148                     To Kill a Mockingbird  ...       3198671   
149                     To Kill a Mockingbird  ...       3198671   

    work_ratings_count  work_text_reviews_count  ratings_1  ratings_2  \
0              4800065                    75867      75504     101676   
1              4800065                    75867      75504     101676   
2              4800065                    75867      75504     101676   
3              4800065                    75867      75504     101676   
4              4800065                    75867      75504     101676   
..                 ...                      ...        ...        ...   
145            3340896                    72586      60427     117415   
146            3340896                    72586      60427     117415   
147            3340896                    72586      60427     117415   
148            3340896                    72586      60427     117415   
149            3340896                    72586      60427     117415   

     ratings_3  ratings_4  ratings_5  user_id  rating  
0       455024    1156318    3011543      314       3  
1       455024    1156318    3011543      588       1  
2       455024    1156318    3011543     2077       2  
3       455024    1156318    3011543     2487       3  
4       455024    1156318    3011543     2900       3  
..         ...        ...        ...      ...     ...  
145     446835    1001952    1714267    19184       5  
146     446835    1001952    1714267    19234       4  
147     446835    1001952    1714267    19248       3  
148     446835    100

In [21]:
bookMerge.isnull().sum()

id                           0
book_id                      0
best_book_id                 0
work_id                      0
books_count                  0
isbn                         0
isbn13                       0
authors                      0
original_publication_year    0
original_title               0
title                        0
language_code                0
average_rating               0
ratings_count                0
work_ratings_count           0
work_text_reviews_count      0
ratings_1                    0
ratings_2                    0
ratings_3                    0
ratings_4                    0
ratings_5                    0
user_id                      0
rating                       0
dtype: int64

==============================================================================================================================

In [22]:
books.sort_values(by = ['average_rating','ratings_count'], ascending = False).head()

id   book_id  best_book_id   work_id  books_count       isbn  \
3627  3628     24812         24812     25599           14  740748475   
861    862  17332218      17332218  16482835           34  765326361   
8853  8854     95602         95602    920037            6  842339523   
4482  4483     24814         24814     25601           21  836221362   
421    422    862041        862041   2962492           76  545044251   

            isbn13            authors  original_publication_year  \
3627  9.780741e+12     Bill Watterson                       2005   
861   9.780765e+12  Brandon Sanderson                       2014   
8853  9.780842e+12    Francine Rivers                       1993   
4482  9.780836e+12     Bill Watterson                       1996   
421   9.780545e+12       J.K. Rowling                       1998   

                                         original_title  ... language_code  \
3627                     The Complete Calvin and Hobbes  ...           eng   
861                                   Words of Radiance  ...           eng   
8853                           Mark of the Lion Trilogy  ...         en-US   
4482  It's a Magical World: A Calvin and Hobbes Coll...  ...           eng   
421                     Complete Harry Potter Boxed Set  ...           eng   

     average_rating  ratings_count  work_ratings_count  \
3627           4.82          28900               29968   
861            4.77          73572              108176   
8853           4.76           9081                9547   
4482           4.75          22351               23429   
421            4.74         190050              204125   

      work_text_reviews_count  ratings_1  ratings_2  ratings_3  ratings_4  \
3627                      861        120        154        693       3117   
861                      7261        154        404       2653      18272   
8853                      731         27         61        323       1363   
4482                      264         36         57        850       3737   
421                      6508       1105       1285       7020      30666   

      ratings_5  
3627      25884  
861       86693  
8853       7773  
4482      18749  
421      164049  

[5 rows x 21 columns]

In [23]:
df_book = books[['original_title', 'average_rating', 'ratings_count']]
df_book

original_title  average_rating  \
0                                      The Hunger Games            4.34   
1              Harry Potter and the Philosopher's Stone            4.44   
2                                              Twilight            3.57   
3                                 To Kill a Mockingbird            4.25   
4                                      The Great Gatsby            3.89   
...                                                 ...             ...   
9994                                 Billy Budd, Sailor            3.09   
9995                                         Bayou Moon            4.09   
9996                                   Means of Ascent             4.25   
9997                              The Mauritius Command            4.35   
9998  Cinderella Ate My Daughter: Dispatches from th...            3.65   

      ratings_count  
0           4780653  
1           4602479  
2           3866839  
3           3198671  
4           2683664  
...             ...  
9994          10866  
9995          17204  
9996          12582  
9997           9421  
9998          11279  

[7860 rows x 3 columns]

In [24]:
df_book.describe()

average_rating  ratings_count
count     7860.000000   7.860000e+03
mean         3.995398   6.117483e+04
std          0.250907   1.751315e+05
min          2.470000   2.773000e+03
25%          3.840000   1.427075e+04
50%          4.010000   2.283800e+04
75%          4.170000   4.625250e+04
max          4.820000   4.780653e+06

Weighted Rating
v is the number of votes for the movie (represented by vote_count)

m is the minimum of votes required to be listed in the chart (to be calculated)

R is the average rating of the movie (represented by vote_average)

C is the mean of average_rating across the whole report (to be calculated)

(v / (v + m) * R) + (m / (m + v) * C)

In [25]:
m = books['ratings_count'].quantile(0.90)

In [26]:
C = books['average_rating'].mean()

In [27]:
# Function that computes the weighted rating of each book
def weighted_rating(x, m = m, C =C ):
    
    v = x['ratings_count']
    R = x['average_rating']
    
    # Calculation based on the IMDB formula
    return (v / (v + m) * R) + (m / (m + v) * C)

In [28]:
m

109322.90000000004

The rating must more than 95%

In [29]:
# Extract all qualified movies into a new DataFrame
qualified_books = df_book.copy().loc[df_book['ratings_count'] >= m]
qualified_books

original_title  average_rating  ratings_count
0                             The Hunger Games            4.34        4780653
1     Harry Potter and the Philosopher's Stone            4.44        4602479
2                                     Twilight            3.57        3866839
3                        To Kill a Mockingbird            4.25        3198671
4                             The Great Gatsby            3.89        2683664
...                                        ...             ...            ...
1069                                 Darkfever            4.14         110812
1139                                  Hex Hall            3.96         109881
1142                         Beautiful Bastard            4.05         111668
1168                                      Onyx            4.33         111448
1452                    Everything, Everything            4.11         132552

[786 rows x 3 columns]

In [30]:
len(qualified_books)

786

Insert weighted rating into columns

In [31]:
qualified_books['weighted_rating'] = qualified_books.apply(weighted_rating, axis=1)


In [32]:
qualified_books.head()

original_title  average_rating  ratings_count  \
0                          The Hunger Games            4.34        4780653   
1  Harry Potter and the Philosopher's Stone            4.44        4602479   
2                                  Twilight            3.57        3866839   
3                     To Kill a Mockingbird            4.25        3198671   
4                          The Great Gatsby            3.89        2683664   

   weighted_rating  
0         4.332296  
1         4.429684  
2         3.581696  
3         4.241586  
4         3.894125

# Recommander Top 10 Book

In [33]:

qualified_books=qualified_books [['original_title', 'average_rating', 'ratings_count', 'weighted_rating']]
qualified_books = qualified_books.sort_values('weighted_rating', ascending = False)

qualified_books.head(10)

original_title  average_rating  ratings_count  \
24        Harry Potter and the Deathly Hallows            4.61        1746574   
26      Harry Potter and the Half-Blood Prince            4.54        1678823   
17    Harry Potter and the Prisoner of Azkaban            4.53        1832823   
23         Harry Potter and the Goblet of Fire            4.53        1753043   
421            Complete Harry Potter Boxed Set            4.74         190050   
134                          A Storm of Swords            4.54         469022   
20   Harry Potter and the Order of the Phoenix            4.46        1735368   
191                       The Name of the Wind            4.55         400101   
1     Harry Potter and the Philosopher's Stone            4.44        4602479   
30                                    The Help            4.45        1531753   

     weighted_rating  
24          4.573796  
26          4.506704  
17          4.499907  
23          4.498618  
421         4.468092  
134         4.437055  
20          4.432466  
191         4.430982  
1           4.429684  
30          4.419716

# Top genres Book

In [34]:
book_tags.head(10)

goodreads_book_id  tag_id   count
0                  1   30574  167697
1                  1   11305   37174
2                  1   11557   34173
3                  1    8717   12986
4                  1   33114   12716
5                  1   11743    9954
6                  1   14017    7169
7                  1    5207    6221
8                  1   22743    4974
9                  1   32989    4364

In [35]:
tags.head(10)

tag_id tag_name
0       0        -
1       1     --1-
2       2    --10-
3       3    --12-
4       4   --122-
5       5   --166-
6       6    --17-
7       7    --19-
8       8     --2-
9       9   --258-

In [36]:
genres = ["Art", "Biography", "Business", "Chick Lit", "Children's", "Christian", "Classics",
          "Comics", "Contemporary", "Cookbooks", "Crime", "Ebooks", "Fantasy", "Fiction",
          "Gay and Lesbian", "Graphic Novels", "Historical Fiction", "History", "Horror",
          "Humor and Comedy", "Manga", "Memoir", "Music", "Mystery", "Nonfiction", "Paranormal",
          "Philosophy", "Poetry", "Psychology", "Religion", "Romance", "Science", "Science Fiction", 
          "Self Help", "Suspense", "Spirituality", "Sports", "Thriller", "Travel", "Young Adult"]

In [37]:
genres = list(map(str.lower, genres))
genres[:4]

['art', 'biography', 'business', 'chick lit']

In [38]:
available_genres = tags.loc[tags.tag_name.str.lower().isin(genres)]

In [39]:
available_genres.head(30)

tag_id      tag_name
2938     2938           art
4605     4605     biography
5951     5951      business
7077     7077     christian
7457     7457      classics
7778     7778        comics
8055     8055  contemporary
8150     8150     cookbooks
8517     8517         crime
10210   10210        ebooks
11305   11305       fantasy
11743   11743       fiction
14552   14552       history
14821   14821        horror
19052   19052         manga
19733   19733        memoir
20673   20673         music
20939   20939       mystery
21773   21773    nonfiction
22973   22973    paranormal
23471   23471    philosophy
23831   23831        poetry
24526   24526    psychology
25647   25647      religion
26138   26138       romance
26816   26816       science
28384   28384  spirituality
28422   28422        sports
29076   29076      suspense
30358   30358      thriller

In [40]:
available_genres_books = book_tags[book_tags.tag_id.isin(available_genres.tag_id)]

In [41]:
print('There are {} books that are tagged with above genres'.format(available_genres_books.shape[0]))

There are 60573 books that are tagged with above genres


In [42]:
available_genres_books.head()

goodreads_book_id  tag_id  count
1                   1   11305  37174
5                   1   11743   9954
25                  1    7457    958
38                  1   22973    673
52                  1   20939    465

In [43]:
available_genres_books['genre'] = available_genres.tag_name.loc[available_genres_books.tag_id].values
available_genres_books.head()

goodreads_book_id  tag_id  count       genre
1                   1   11305  37174     fantasy
5                   1   11743   9954     fiction
25                  1    7457    958    classics
38                  1   22973    673  paranormal
52                  1   20939    465     mystery

In [44]:
books.head()

id  book_id  best_book_id  work_id  books_count       isbn        isbn13  \
0   1  2767052       2767052  2792775          272  439023483  9.780439e+12   
1   2        3             3  4640799          491  439554934  9.780440e+12   
2   3    41865         41865  3212258          226  316015849  9.780316e+12   
3   4     2657          2657  3275794          487   61120081  9.780061e+12   
4   5     4671          4671   245494         1356  743273567  9.780743e+12   

                       authors  original_publication_year  \
0              Suzanne Collins                       2008   
1  J.K. Rowling, Mary GrandPré                       1997   
2              Stephenie Meyer                       2005   
3                   Harper Lee                       1960   
4          F. Scott Fitzgerald                       1925   

                             original_title  ... language_code average_rating  \
0                          The Hunger Games  ...           eng           4.34   
1  Harry Potter and the Philosopher's Stone  ...           eng           4.44   
2                                  Twilight  ...         en-US           3.57   
3                     To Kill a Mockingbird  ...           eng           4.25   
4                          The Great Gatsby  ...           eng           3.89   

   ratings_count  work_ratings_count  work_text_reviews_count  ratings_1  \
0        4780653             4942365                   155254      66715   
1        4602479             4800065                    75867      75504   
2        3866839             3916824                    95009     456191   
3        3198671             3340896                    72586      60427   
4        2683664             2773745                    51992      86236   

   ratings_2  ratings_3  ratings_4  ratings_5  
0     127936     560092    1481305    2706317  
1     101676     455024    1156318    3011543  
2     436802     793319     875073    1355439  
3     117415     446835    1001952    1714267  
4     197621     606158     936012     947718  

[5 rows x 21 columns]

In [45]:
def recommender_genre(genre, percentile=0.85):
    df = available_genres_books[available_genres_books['genre'] == genre.lower()]
    #sets the 'book_id' column as the index of the books DataFrame. The index is the row labels of a DataFrame,
    #and setting a column as the index can be useful for faster lookups, merging, or joining operations.
    books_indexed = books.set_index('book_id')
    
    # Filter out book IDs not present in the 'books' DataFrame
    valid_book_ids = df.goodreads_book_id[df.goodreads_book_id.isin(books_indexed.index)]
    
    qualified = books_indexed.loc[valid_book_ids]
    
    v = qualified['ratings_count']
    m = qualified['ratings_count'].quantile(percentile)
    R = qualified['average_rating']
    C = qualified['average_rating'].mean()
    qualified['weighted_rating'] = (v / (v + m) * R) + (m / (m + v) * C)

    qualified.sort_values('weighted_rating', ascending=False, inplace=True)
    return qualified

In [46]:
available_genres_books

goodreads_book_id  tag_id  count         genre
1                       1   11305  37174       fantasy
5                       1   11743   9954       fiction
25                      1    7457    958      classics
38                      1   22973    673    paranormal
52                      1   20939    465       mystery
...                   ...     ...    ...           ...
999796           32848471    8055      8  contemporary
999814           33288638   26138    385       romance
999816           33288638    8055    216  contemporary
999831           33288638   28422     30        sports
999842           33288638   11743     21       fiction

[60573 rows x 4 columns]

In [47]:
cols = ['title','authors','original_publication_year','average_rating','ratings_count','work_text_reviews_count','weighted_rating']

In [48]:
genre = 'Fiction'
recommender_genre(genre)[cols].head(15)

title  \
book_id                                                      
136251   Harry Potter and the Deathly Hallows (Harry Po...   
862041            Harry Potter Boxset (Harry Potter, #1-7)   
1        Harry Potter and the Half-Blood Prince (Harry ...   
5        Harry Potter and the Prisoner of Azkaban (Harr...   
6        Harry Potter and the Goblet of Fire (Harry Pot...   
62291       A Storm of Swords (A Song of Ice and Fire, #3)   
186074   The Name of the Wind (The Kingkiller Chronicle...   
2        Harry Potter and the Order of the Phoenix (Har...   
18512    The Return of the King (The Lord of the Rings,...   
3        Harry Potter and the Sorcerer's Stone (Harry P...   
1215032  The Wise Man's Fear (The Kingkiller Chronicle,...   
4667024                                           The Help   
13496       A Game of Thrones (A Song of Ice and Fire, #1)   
4502507  The Last Olympian (Percy Jackson and the Olymp...   
7235533      The Way of Kings (The Stormlight Archive, #1)   

                                         authors  original_publication_year  \
book_id                                                                       
136251               J.K. Rowling, Mary GrandPré                       2007   
862041                              J.K. Rowling                       1998   
1                    J.K. Rowling, Mary GrandPré                       2005   
5        J.K. Rowling, Mary GrandPré, Rufus Beck                       1999   
6                    J.K. Rowling, Mary GrandPré                       2000   
62291                         George R.R. Martin                       2000   
186074                          Patrick Rothfuss                       2007   
2                    J.K. Rowling, Mary GrandPré                       2003   
18512                             J.R.R. Tolkien                       1955   
3                    J.K. Rowling, Mary GrandPré                       1997   
1215032                         Patrick Rothfuss                       2011   
4667024                         Kathryn Stockett                       2009   
13496                         George R.R. Martin                       1996   
4502507                             Rick Riordan                       2009   
7235533                        Brandon Sanderson                       2010   

         average_rating  ratings_count  work_text_reviews_count  \
book_id                                                           
136251             4.61        1746574                    51942   
862041             4.74         190050                     6508   
1                  4.54        1678823                    27520   
5                  4.53        1832823                    36099   
6                  4.53        1753043                    31084   
62291              4.54         469022                    19497   
186074             4.55         400101                    28631   
2                  4.46        1735368                    28685   
18512              4.51         463959                     6644   
3                  4.44        4602479                    75867   
1215032            4.57         245686                    15503   
4667024            4.45        1531753                    78204   
13496              4.45        1319204                    46205   
4502507            4.50         397500                    17693   
7235533            4.64         144822                    10947   

         weighted_rating  
book_id                   
136251          4.583990  
862041          4.524886  
1               4.516053  
5               4.508387  
6               4.507444  
62291           4.462986  
186074          4.460246  
2               4.440195  
18512           4.436531  
3               4.432659  
1215032         4.432516  
4667024         4.428167  
13496           4.424843  
4502507         4.417860  
7235533         4.415696

In [49]:
list(enumerate(available_genres.tag_name))

[(0, 'art'),
 (1, 'biography'),
 (2, 'business'),
 (3, 'christian'),
 (4, 'classics'),
 (5, 'comics'),
 (6, 'contemporary'),
 (7, 'cookbooks'),
 (8, 'crime'),
 (9, 'ebooks'),
 (10, 'fantasy'),
 (11, 'fiction'),
 (12, 'history'),
 (13, 'horror'),
 (14, 'manga'),
 (15, 'memoir'),
 (16, 'music'),
 (17, 'mystery'),
 (18, 'nonfiction'),
 (19, 'paranormal'),
 (20, 'philosophy'),
 (21, 'poetry'),
 (22, 'psychology'),
 (23, 'religion'),
 (24, 'romance'),
 (25, 'science'),
 (26, 'spirituality'),
 (27, 'sports'),
 (28, 'suspense'),
 (29, 'thriller'),
 (30, 'travel')]

In [50]:
idx = 24  # romance
recommender_genre(list(available_genres.tag_name)[idx])[cols].head(15)

title  \
book_id                                                       
136251    Harry Potter and the Deathly Hallows (Harry Po...   
1         Harry Potter and the Half-Blood Prince (Harry ...   
862041             Harry Potter Boxset (Harry Potter, #1-7)   
62291        A Storm of Swords (A Song of Ice and Fire, #3)   
186074    The Name of the Wind (The Kingkiller Chronicle...   
2         Harry Potter and the Order of the Phoenix (Har...   
1215032   The Wise Man's Fear (The Kingkiller Chronicle,...   
13496        A Game of Thrones (A Song of Ice and Fire, #1)   
4502507   The Last Olympian (Percy Jackson and the Olymp...   
21853621                                    The Nightingale   
18335634      Clockwork Princess (The Infernal Devices, #3)   
10025305        Clockwork Prince (The Infernal Devices, #2)   
12127750     The Mark of Athena (The Heroes of Olympus, #3)   
17332218     Words of Radiance (The Stormlight Archive, #2)   
12127810     The House of Hades (The Heroes of Olympus, #4)   

                              authors  original_publication_year  \
book_id                                                            
136251    J.K. Rowling, Mary GrandPré                       2007   
1         J.K. Rowling, Mary GrandPré                       2005   
862041                   J.K. Rowling                       1998   
62291              George R.R. Martin                       2000   
186074               Patrick Rothfuss                       2007   
2         J.K. Rowling, Mary GrandPré                       2003   
1215032              Patrick Rothfuss                       2011   
13496              George R.R. Martin                       1996   
4502507                  Rick Riordan                       2009   
21853621               Kristin Hannah                       2015   
18335634              Cassandra Clare                       2013   
10025305              Cassandra Clare                       2011   
12127750                 Rick Riordan                       2012   
17332218            Brandon Sanderson                       2014   
12127810                 Rick Riordan                       2013   

          average_rating  ratings_count  work_text_reviews_count  \
book_id                                                            
136251              4.61        1746574                    51942   
1                   4.54        1678823                    27520   
862041              4.74         190050                     6508   
62291               4.54         469022                    19497   
186074              4.55         400101                    28631   
2                   4.46        1735368                    28685   
1215032             4.57         245686                    15503   
13496               4.45        1319204                    46205   
4502507             4.50         397500                    17693   
21853621            4.54         253606                    37279   
18335634            4.59         162099                    23261   
10025305            4.46         307994                    17854   
12127750            4.47         277491                    13332   
17332218            4.77          73572                     7261   
12127810            4.54         159774                    11843   

          weighted_rating  
book_id                    
136251           4.581879  
1                4.514100  
862041           4.512087  
62291            4.457348  
186074           4.453839  
2                4.438561  
1215032          4.423623  
13496            4.422791  
4502507          4.411965  
21853621         4.404470  
18335634         4.386947  
10025305         4.360317  
12127750         4.359580  
17332218         4.356740  
12127810         4.352112

In [51]:
list(enumerate(available_genres.tag_name))

[(0, 'art'),
 (1, 'biography'),
 (2, 'business'),
 (3, 'christian'),
 (4, 'classics'),
 (5, 'comics'),
 (6, 'contemporary'),
 (7, 'cookbooks'),
 (8, 'crime'),
 (9, 'ebooks'),
 (10, 'fantasy'),
 (11, 'fiction'),
 (12, 'history'),
 (13, 'horror'),
 (14, 'manga'),
 (15, 'memoir'),
 (16, 'music'),
 (17, 'mystery'),
 (18, 'nonfiction'),
 (19, 'paranormal'),
 (20, 'philosophy'),
 (21, 'poetry'),
 (22, 'psychology'),
 (23, 'religion'),
 (24, 'romance'),
 (25, 'science'),
 (26, 'spirituality'),
 (27, 'sports'),
 (28, 'suspense'),
 (29, 'thriller'),
 (30, 'travel')]

# Top 20 Gerne Book

In [52]:
idx = int(input("choose one genres : "))
recommender_genre(list(available_genres.tag_name)[idx])[cols].head(20)

choose one genres : 24


title  \
book_id                                                       
136251    Harry Potter and the Deathly Hallows (Harry Po...   
1         Harry Potter and the Half-Blood Prince (Harry ...   
862041             Harry Potter Boxset (Harry Potter, #1-7)   
62291        A Storm of Swords (A Song of Ice and Fire, #3)   
186074    The Name of the Wind (The Kingkiller Chronicle...   
2         Harry Potter and the Order of the Phoenix (Har...   
1215032   The Wise Man's Fear (The Kingkiller Chronicle,...   
13496        A Game of Thrones (A Song of Ice and Fire, #1)   
4502507   The Last Olympian (Percy Jackson and the Olymp...   
21853621                                    The Nightingale   
18335634      Clockwork Princess (The Infernal Devices, #3)   
10025305        Clockwork Prince (The Infernal Devices, #2)   
12127750     The Mark of Athena (The Heroes of Olympus, #3)   
17332218     Words of Radiance (The Stormlight Archive, #2)   
12127810     The House of Hades (The Heroes of Olympus, #4)   
9520360      The Son of Neptune (The Heroes of Olympus, #2)   
2767052             The Hunger Games (The Hunger Games, #1)   
2120932   The Battle of the Labyrinth (Percy Jackson and...   
8755785   City of Heavenly Fire (The Mortal Instruments,...   
18006496             Queen of Shadows (Throne of Glass, #4)   

                              authors  original_publication_year  \
book_id                                                            
136251    J.K. Rowling, Mary GrandPré                       2007   
1         J.K. Rowling, Mary GrandPré                       2005   
862041                   J.K. Rowling                       1998   
62291              George R.R. Martin                       2000   
186074               Patrick Rothfuss                       2007   
2         J.K. Rowling, Mary GrandPré                       2003   
1215032              Patrick Rothfuss                       2011   
13496              George R.R. Martin                       1996   
4502507                  Rick Riordan                       2009   
21853621               Kristin Hannah                       2015   
18335634              Cassandra Clare                       2013   
10025305              Cassandra Clare                       2011   
12127750                 Rick Riordan                       2012   
17332218            Brandon Sanderson                       2014   
12127810                 Rick Riordan                       2013   
9520360                  Rick Riordan                       2011   
2767052               Suzanne Collins                       2008   
2120932                  Rick Riordan                       2008   
8755785               Cassandra Clare                       2014   
18006496                Sarah J. Maas                       2015   

          average_rating  ratings_count  work_text_reviews_count  \
book_id                                                            
136251              4.61        1746574                    51942   
1                   4.54        1678823                    27520   
862041              4.74         190050                     6508   
62291               4.54         469022                    19497   
186074              4.55         400101                    28631   
2                   4.46        1735368                    28685   
1215032             4.57         245686                    15503   
13496               4.45        1319204                    46205   
4502507             4.50         397500                    17693   
21853621            4.54         253606                    37279   
18335634            4.59         162099                    23261   
10025305            4.46         307994                    17854   
12127750            4.47         277491                    13332   
17332218            4.77          73572                     7261   
12127810            4.54         159774                    11843   
9520360       

# Content Based Filtering

In [53]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [54]:
books = pd.read_csv('dataset/books.csv', low_memory=False)
books.head()

id  book_id  best_book_id  work_id  books_count       isbn        isbn13  \
0   1  2767052       2767052  2792775          272  439023483  9.780439e+12   
1   2        3             3  4640799          491  439554934  9.780440e+12   
2   3    41865         41865  3212258          226  316015849  9.780316e+12   
3   4     2657          2657  3275794          487   61120081  9.780061e+12   
4   5     4671          4671   245494         1356  743273567  9.780743e+12   

                       authors  original_publication_year  \
0              Suzanne Collins                     2008.0   
1  J.K. Rowling, Mary GrandPré                     1997.0   
2              Stephenie Meyer                     2005.0   
3                   Harper Lee                     1960.0   
4          F. Scott Fitzgerald                     1925.0   

                             original_title  ... ratings_count  \
0                          The Hunger Games  ...       4780653   
1  Harry Potter and the Philosopher's Stone  ...       4602479   
2                                  Twilight  ...       3866839   
3                     To Kill a Mockingbird  ...       3198671   
4                          The Great Gatsby  ...       2683664   

  work_ratings_count  work_text_reviews_count  ratings_1  ratings_2  \
0            4942365                   155254      66715     127936   
1            4800065                    75867      75504     101676   
2            3916824                    95009     456191     436802   
3            3340896                    72586      60427     117415   
4            2773745                    51992      86236     197621   

   ratings_3  ratings_4  ratings_5  \
0     560092    1481305    2706317   
1     455024    1156318    3011543   
2     793319     875073    1355439   
3     446835    1001952    1714267   
4     606158     936012     947718   

                                           image_url  \
0  https://images.gr-assets.com/books/1447303603m...   
1  https://images.gr-assets.com/books/1474154022m...   
2  https://images.gr-assets.com/books/1361039443m...   
3  https://images.gr-assets.com/books/1361975680m...   
4  https://images.gr-assets.com/books/1490528560m...   

                                     small_image_url  
0  https://images.gr-assets.com/books/1447303603s...  
1  https://images.gr-assets.com/books/1474154022s...  
2  https://images.gr-assets.com/books/1361039443s...  
3  https://images.gr-assets.com/books/1361975680s...  
4  https://images.gr-assets.com/books/1490528560s...  

[5 rows x 23 columns]

In [55]:
ratings = pd.read_csv('dataset/ratings.csv', low_memory=False)
ratings.head()

book_id  user_id  rating
0        1      314       5
1        1      439       3
2        1      588       5
3        1     1169       4
4        1     1185       4

In [56]:
tags = pd.read_csv('dataset/tags.csv', low_memory=False)
tags.head()

tag_id tag_name
0       0        -
1       1     --1-
2       2    --10-
3       3    --12-
4       4   --122-

In [57]:
book_tags = pd.read_csv('dataset/book_tags.csv', low_memory=False)
book_tags.head()

goodreads_book_id  tag_id   count
0                  1   30574  167697
1                  1   11305   37174
2                  1   11557   34173
3                  1    8717   12986
4                  1   33114   12716

In [58]:
books.columns

Index(['id', 'book_id', 'best_book_id', 'work_id', 'books_count', 'isbn',
       'isbn13', 'authors', 'original_publication_year', 'original_title',
       'title', 'language_code', 'average_rating', 'ratings_count',
       'work_ratings_count', 'work_text_reviews_count', 'ratings_1',
       'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5', 'image_url',
       'small_image_url'],
      dtype='object')

# Content Based Filtering Recommender System

Filter the book recommendation through similar author

In [59]:
books_authors = books[['book_id', 'title','authors']]
books_authors.head()

book_id                                              title  \
0  2767052            The Hunger Games (The Hunger Games, #1)   
1        3  Harry Potter and the Sorcerer's Stone (Harry P...   
2    41865                            Twilight (Twilight, #1)   
3     2657                              To Kill a Mockingbird   
4     4671                                   The Great Gatsby   

                       authors  
0              Suzanne Collins  
1  J.K. Rowling, Mary GrandPré  
2              Stephenie Meyer  
3                   Harper Lee  
4          F. Scott Fitzgerald

In [60]:
# Remove words such as is, an, the, etc
tf = TfidfVectorizer(analyzer='word', stop_words='english')
# Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tf.fit_transform(books_authors['authors'])
# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [61]:
# Build  1-dimensional array for titles
titles = books_authors['title']
indices = pd.Series(books_authors.index, index=books_authors['title'])

In [62]:
# Function to get book recommendations based on the cosine similarity score of authors
def authors_recommendations(title):
    # Get the index of the matched book title
    idx = indices[title]
    # Get the pairwise similarity scores of all books with the selected
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Sort the books based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores of the 10 most similar books
    sim_scores = sim_scores[1:11]
    # Get the book indices
    book_indices = [i[0] for i in sim_scores]
    # Return the top 10 similar books
    return titles.iloc[book_indices]

In [63]:
# Getting input of book title from user
book_name = input('Enter the book title: ')

Enter the book title: Gone Girl


In [64]:
authors_recommendations(book_name)

225                              Dark Places
243                            Sharp Objects
2168                             The Grownup
9758     Another Bullshit Night in Suck City
2058      American Assassin (Mitch Rapp, #1)
2514         Act of Treason (Mitch Rapp, #9)
2621           Memorial Day (Mitch Rapp, #7)
2829        Consent to Kill (Mitch Rapp, #8)
2834    Separation of Power (Mitch Rapp, #5)
2919      Transfer of Power (Mitch Rapp, #3)
Name: title, dtype: object

In [65]:
# Combine book tags with tags.csv
join_tags = pd.merge(book_tags, tags, left_on='tag_id', right_on='tag_id', how='inner')
join_tags.head()

goodreads_book_id  tag_id   count tag_name
0                  1   30574  167697  to-read
1                  2   30574   24549  to-read
2                  3   30574  496107  to-read
3                  5   30574   11909  to-read
4                  6   30574     298  to-read

In [66]:
# Combine file
books_merge_tags = pd.merge(books_authors, join_tags, left_on='book_id', right_on='goodreads_book_id', how='inner')
books_merge_tags.head()

book_id                                    title          authors  \
0  2767052  The Hunger Games (The Hunger Games, #1)  Suzanne Collins   
1  2767052  The Hunger Games (The Hunger Games, #1)  Suzanne Collins   
2  2767052  The Hunger Games (The Hunger Games, #1)  Suzanne Collins   
3  2767052  The Hunger Games (The Hunger Games, #1)  Suzanne Collins   
4  2767052  The Hunger Games (The Hunger Games, #1)  Suzanne Collins   

   goodreads_book_id  tag_id  count           tag_name  
0            2767052   30574  11314            to-read  
1            2767052   11305  10836            fantasy  
2            2767052   11557  50755          favorites  
3            2767052    8717  35418  currently-reading  
4            2767052   33114  25968        young-adult

In [67]:
# Define a TF-IDF Vectorizer Object.
# Remove words such as 'the', 'a'
# Convert text to feature vectors that can be used as input to estimator
tf1 = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix1 = tf1.fit_transform(books_merge_tags['tag_name'].head(10000))
cosine_sim1 = linear_kernel(tfidf_matrix1, tfidf_matrix1)
cosine_sim1

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [68]:
# Build  1-dimensional array with book titles
book_titles = books_authors['title']
book_indices = pd.Series(books.index, index=books_authors['title'])

# Function that get book recommendations based on the cosine similarity score of book tags
def tags_recommendations(title):
    # Get the index of the matched book title
    idx = book_indices[title]
    # Get the pairwise similarity scores of all books with the selected
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Sort the books based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores of the 10 most similar books
    sim_scores = sim_scores[1:11]
    # Get the book indices
    book_indices1 = [i[0] for i in sim_scores]
    # Return the top 10 similar books
    return titles.iloc[book_indices1]

In [69]:
book_name1 = input('Enter the book title:  ')

Enter the book title:  Gone Girl


In [70]:
tags_recommendations(book_name1).head(10)

225                              Dark Places
243                            Sharp Objects
2168                             The Grownup
9758     Another Bullshit Night in Suck City
2058      American Assassin (Mitch Rapp, #1)
2514         Act of Treason (Mitch Rapp, #9)
2621           Memorial Day (Mitch Rapp, #7)
2829        Consent to Kill (Mitch Rapp, #8)
2834    Separation of Power (Mitch Rapp, #5)
2919      Transfer of Power (Mitch Rapp, #3)
Name: title, dtype: object

## Recommendation of books using combination of attributes(author and tags) for better results.


In [71]:
temp_df = books_merge_tags.groupby('book_id')['tag_name'].apply(' '.join).reset_index()
temp_df.head()

book_id                                           tag_name
0        1  to-read fantasy favorites currently-reading yo...
1        2  to-read fantasy favorites currently-reading yo...
2        3  to-read fantasy favorites currently-reading yo...
3        5  to-read fantasy favorites currently-reading yo...
4        6  to-read fantasy young-adult fiction harry-pott...

In [72]:
books_authors = pd.merge(books_authors, temp_df, left_on='book_id', right_on='book_id', how='inner')
books_authors.head()

book_id                                              title  \
0  2767052            The Hunger Games (The Hunger Games, #1)   
1        3  Harry Potter and the Sorcerer's Stone (Harry P...   
2    41865                            Twilight (Twilight, #1)   
3     2657                              To Kill a Mockingbird   
4     4671                                   The Great Gatsby   

                       authors  \
0              Suzanne Collins   
1  J.K. Rowling, Mary GrandPré   
2              Stephenie Meyer   
3                   Harper Lee   
4          F. Scott Fitzgerald   

                                            tag_name  
0  to-read fantasy favorites currently-reading yo...  
1  to-read fantasy favorites currently-reading yo...  
2  to-read fantasy favorites currently-reading yo...  
3  to-read favorites currently-reading young-adul...  
4  to-read favorites currently-reading young-adul...

In [73]:
books_authors['collection'] = (pd.Series(books_authors[['authors', 'tag_name']]
                      .fillna('')
                      .values.tolist()
                      ).str.join(' '))

In [74]:
tf_coll = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix_coll = tf_coll.fit_transform(books_authors['collection'])
cosine_sim_coll = linear_kernel(tfidf_matrix_coll, tfidf_matrix_coll)

# Build a 1-dimensional array with book titles
titles = books_authors['title']
indices = pd.Series(books_authors.index, index=books_authors['title'])

# Get book recommendations based on the cosine similarity score of books tags
def collection_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim_coll[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    book_indices = [i[0] for i in sim_scores]
    return titles.iloc[book_indices]

In [75]:
book_title = input('Enter the book title: ')

Enter the book title: Gone Girl


In [76]:
collection_recommendations(book_title).head(10)

243              Sharp Objects
225                Dark Places
2168               The Grownup
2616    The Kind Worth Killing
3311               Pretty Baby
1344              Pretty Girls
810              The Good Girl
7870               Second Life
60       The Girl on the Train
444       Before I Go to Sleep
Name: title, dtype: object

## Use CountVectorizer to perform recommendation based on authors

In [77]:
book_authors = books[['book_id', 'title','authors']]
book_authors.head()

book_id                                              title  \
0  2767052            The Hunger Games (The Hunger Games, #1)   
1        3  Harry Potter and the Sorcerer's Stone (Harry P...   
2    41865                            Twilight (Twilight, #1)   
3     2657                              To Kill a Mockingbird   
4     4671                                   The Great Gatsby   

                       authors  
0              Suzanne Collins  
1  J.K. Rowling, Mary GrandPré  
2              Stephenie Meyer  
3                   Harper Lee  
4          F. Scott Fitzgerald

In [78]:
book_titles = input('Enter the book title: ')

Enter the book title: Gone Girl


In [79]:
# Initialize vectorizer
# Min_df = rare words, max_df = most used words
vect1 = CountVectorizer(analyzer = 'word', ngram_range = (1,2), stop_words = 'english', min_df = 0.002)

In [80]:
# Fit into the authors
vect1.fit(book_authors['authors'])
title_matrix = vect1.transform(book_authors['authors'])

In [81]:
#features= vect1.get_feature_names() ver problem
features = vect1.get_feature_names_out()
features

array(['abbi', 'abbi glines', 'agatha', 'agatha christie', 'alan', 'alex',
       'alexander', 'alice', 'amy', 'anderson', 'andrew', 'andrews',
       'anita', 'ann', 'anna', 'anne', 'anne rice', 'anthony', 'archer',
       'armentrout', 'armstrong', 'arthur', 'ashley', 'asimov',
       'baldacci', 'banks', 'barbara', 'ben', 'bernard', 'blake',
       'brandon', 'brandon sanderson', 'brett', 'brian', 'brian vaughan',
       'briggs', 'brooks', 'brown', 'bryan', 'butcher', 'cabot', 'card',
       'carl', 'charlaine', 'charlaine harris', 'charles', 'charlie',
       'child', 'chris', 'christie', 'christopher', 'chuck', 'clare',
       'clark', 'clarke', 'clive', 'coben', 'cole', 'colleen', 'connelly',
       'cornwell', 'dan', 'daniel', 'dave', 'david', 'david baldacci',
       'david eddings', 'dean', 'dean koontz', 'diana', 'don', 'donald',
       'douglas', 'douglas preston', 'doyle', 'dr', 'dr seuss', 'eddings',
       'edward', 'elizabeth', 'ellen', 'ellis', 'emily', 'eric', 'erin',

In [82]:
cosine_sim_titles = cosine_similarity(title_matrix, title_matrix)
cosine_sim_titles

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [83]:
indices1 = pd.Series(book_authors.index,index=book_authors['title'])
title_id = indices1[book_titles]

In [84]:
# Find out what features have been considered  by the vectorizer for a given title 
# Squeeze activity matrix into array
feature_array = np.squeeze(title_matrix[title_id].toarray())
idx = np.where(feature_array > 0)

In [85]:
# Cosine similarity with other similar titles and determine the total number of books to recommend
n = 11 
top_n_idx = np.flip(np.argsort(cosine_sim_titles[title_id,]), axis = 0)[0:n]
top_n_sim_values = cosine_sim_titles[title_id, top_n_idx]

In [86]:
# Find top n with values > 0
top_n_idx = top_n_idx[top_n_sim_values > 0]
scores = top_n_sim_values[top_n_sim_values > 0]

In [87]:
book_authors['title'].iloc[title_id]
pd.DataFrame({"books": book_authors['title'].iloc[top_n_idx].values,
           "scores":scores}, columns = ["books","scores"])

books  scores
0     Pursuit of Honor (Mitch Rapp, #12)     1.0
1        Act of Treason (Mitch Rapp, #9)     1.0
2                              Gone Girl     1.0
3         The Survivor (Mitch Rapp, #14)     1.0
4     Extreme Measures (Mitch Rapp, #11)     1.0
5     American Assassin (Mitch Rapp, #1)     1.0
6     Transfer of Power (Mitch Rapp, #3)     1.0
7                            Dark Places     1.0
8   Protect and Defend (Mitch Rapp, #10)     1.0
9       Executive Power (Mitch Rapp, #6)     1.0
10  Separation of Power (Mitch Rapp, #5)     1.0

## Count vector for tag and author recommendation

In [88]:
book_nama = input('Enter the book title: ')

Enter the book title: Gone Girl


In [89]:
vect1 = CountVectorizer(analyzer = 'word', ngram_range = (1,2), stop_words = 'english', min_df = 0.002)

In [90]:
vect1.fit(books_authors['collection'])
title_matrix = vect1.transform(books_authors['collection'])

In [91]:

#features1= vect1.get_feature_names() ver problem
features1 = vect1.get_feature_names_out()
features

array(['abbi', 'abbi glines', 'agatha', 'agatha christie', 'alan', 'alex',
       'alexander', 'alice', 'amy', 'anderson', 'andrew', 'andrews',
       'anita', 'ann', 'anna', 'anne', 'anne rice', 'anthony', 'archer',
       'armentrout', 'armstrong', 'arthur', 'ashley', 'asimov',
       'baldacci', 'banks', 'barbara', 'ben', 'bernard', 'blake',
       'brandon', 'brandon sanderson', 'brett', 'brian', 'brian vaughan',
       'briggs', 'brooks', 'brown', 'bryan', 'butcher', 'cabot', 'card',
       'carl', 'charlaine', 'charlaine harris', 'charles', 'charlie',
       'child', 'chris', 'christie', 'christopher', 'chuck', 'clare',
       'clark', 'clarke', 'clive', 'coben', 'cole', 'colleen', 'connelly',
       'cornwell', 'dan', 'daniel', 'dave', 'david', 'david baldacci',
       'david eddings', 'dean', 'dean koontz', 'diana', 'don', 'donald',
       'douglas', 'douglas preston', 'doyle', 'dr', 'dr seuss', 'eddings',
       'edward', 'elizabeth', 'ellen', 'ellis', 'emily', 'eric', 'erin',

In [92]:
cosine_sim_titles1 = cosine_similarity(title_matrix, title_matrix)

In [93]:
indices2 = pd.Series(books_authors.index,index=books_authors['title'])
title_id1 = indices2[book_nama]

In [94]:
feature_array = np.squeeze(title_matrix[title_id1].toarray()) #squeeze activity matrix into array
idx = np.where(feature_array > 0)

In [95]:
# Number of books to show
n = 11
top_n_idx1 = np.flip(np.argsort(cosine_sim_titles1[title_id1,]), axis = 0)[0:n]
top_n_sim_values1 = cosine_sim_titles1[title_id1, top_n_idx1]

In [96]:
# Find top n with values > 0
top_n_idx1 = top_n_idx1[top_n_sim_values1 > 0]
scores1 = top_n_sim_values1[top_n_sim_values1 > 0]

In [97]:
books_authors['title'].iloc[title_id1]
pd.DataFrame({"books": books_authors['title'].iloc[top_n_idx1].values,
           "scores":scores1}, columns = ["books","scores"])

books    scores
0               Gone Girl  1.000000
1    Where They Found Her  0.800593
2             Pretty Baby  0.799111
3           Sharp Objects  0.795383
4           The Good Girl  0.794806
5              Disclaimer  0.794438
6   The Girl on the Train  0.792013
7             Dark Places  0.790939
8   Every Fifteen Minutes  0.790613
9                 Descent  0.787350
10            Those Girls  0.787056